## 1. The World Bank's international debt data
<p>No es que los humanos solo tengamos deudas para administrar nuestras necesidades. Un país también puede endeudarse para administrar su economía. Por ejemplo, el gasto en infraestructura es un ingrediente costoso requerido para que los ciudadanos de un país lleven una vida cómoda. El Banco Mundial es la organización que proporciona deuda a los países.</p>

<!-- <p>En este notebook, vamos a analizar los datos de la deuda internacional recopilados por el Banco Mundial. El conjunto de datos contiene información sobre el monto de la deuda (en USD) que deben los países en desarrollo en varias categorías.</p>  -->
    

"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD
"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD
"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD
"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD
"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD
"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD
"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD
"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD
"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD
"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD
"PPG, official creditors (INT, current US$)",DT.INT.OFFT.CD
"Principal repayments on external debt, long-term (AMT, current US$)",DT.AMT.DLXF.CD
"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",DT.INT.DPNG.CD
"PPG, bonds (AMT, current US$)",DT.AMT.PBND.CD
"PPG, bonds (INT, current US$)",DT.INT.PBND.CD
"PPG, commercial banks (AMT, current US$)",DT.AMT.PCBK.CD
"PPG, commercial banks (DIS, current US$)",DT.DIS.PCBK.CD
"PPG, commercial banks (INT, current US$)",DT.INT.PCBK.CD
"PPG, other private creditors (AMT, current US$)",DT.AMT.PROP.CD
"PPG, other private creditors (DIS, current US$)",DT.DIS.PROP.CD
"PPG, other private creditors (INT, current US$)",DT.INT.PROP.CD
"PPG, private creditors (AMT, current US$)",DT.AMT.PRVT.CD
"PPG, private creditors (DIS, current US$)",DT.DIS.PRVT.CD
"PPG, private creditors (INT, current US$)",DT.INT.PRVT.CD
"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",DT.AMT.DPNG.CD

<p>Vamos a encontrar las respuestas a preguntas como:

<p>¿Cuál es el monto total de la deuda que deben los países enumerados en el conjunto de datos?
<p>¿Qué país posee la cantidad máxima de deuda y cómo se ve esa cantidad?
<p>¿Cuál es el monto promedio de la deuda de los países a través de diferentes indicadores de deuda?
    
Además tenemos otro dataset en el que encontramos información histórica de algunos índices de desarrollo, entre los que se encuentran algunos de deuda como son:

Series Name,Series Code
"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN

"Central government debt, total (current LCU)",GC.DOD.TOTL.CN

"Central government debt, total (% of GDP)",GC.DOD.TOTL.GD.ZS

#### 1. Inicializar y cargar el contexto spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz #d
!pip install -q findspark 

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

A continuación se cargarán los datasets:

In [2]:
desa = sc.textFile("indicadores_desarrollo.csv")
desa.take(10)

['country_name,country_code,anho,GC.DOD.TOTL.CN,GC.DOD.TOTL.GD.ZS,SP.DYN.CBRT.IN',
 'Albania,ALB,2014 [YR2014],1023040800000,73.3202469198429,12.259',
 'Albania,ALB,2015 [YR2015],1145500940000,79.8644448747259,12.197',
 'Albania,ALB,2016 [YR2016],1188814490000,80.7355748396923,12.08',
 'Australia,AUS,2014 [YR2014],674700000,0.042207528166503,13.2',
 'Australia,AUS,2015 [YR2015],762718000,0.0469480198522591,12.9',
 'Australia,AUS,2016 [YR2016],907555000,0.054648482520169,12.9',
 '"Bahamas, The",BHS,2014 [YR2014],5158420000,47.2672793747079,14.34',
 '"Bahamas, The",BHS,2015 [YR2015],5655150000,48.1195170307089,14.224',
 '"Bahamas, The",BHS,2016 [YR2016],6138814000,51.4207431481606,14.131']

In [3]:
desa_sep = desa.map(lambda x: x.split(",")) #separamos por la coma
header = desa_sep.first()
desa_sep = desa_sep.filter(lambda x: x != header) #eliminamos la primera fila, que hace referencia al encabezado
desa_sep.take(5)

[['Albania',
  'ALB',
  '2014 [YR2014]',
  '1023040800000',
  '73.3202469198429',
  '12.259'],
 ['Albania',
  'ALB',
  '2015 [YR2015]',
  '1145500940000',
  '79.8644448747259',
  '12.197'],
 ['Albania',
  'ALB',
  '2016 [YR2016]',
  '1188814490000',
  '80.7355748396923',
  '12.08'],
 ['Australia',
  'AUS',
  '2014 [YR2014]',
  '674700000',
  '0.042207528166503',
  '13.2'],
 ['Australia',
  'AUS',
  '2015 [YR2015]',
  '762718000',
  '0.0469480198522591',
  '12.9']]

In [4]:
deuda = sc.textFile("indicadores_deuda.csv")
deuda.take(5)

['country_name,country_code,DT.AMT.BLAT.CD,DT.AMT.DLXF.CD,DT.AMT.DPNG.CD,DT.AMT.MLAT.CD,DT.AMT.OFFT.CD,DT.AMT.PBND.CD,DT.AMT.PCBK.CD,DT.AMT.PROP.CD,DT.AMT.PRVT.CD,DT.DIS.BLAT.CD,DT.DIS.DLXF.CD,DT.DIS.MLAT.CD,DT.DIS.OFFT.CD,DT.DIS.PCBK.CD,DT.DIS.PROP.CD,DT.DIS.PRVT.CD,DT.INT.BLAT.CD,DT.INT.DLXF.CD,DT.INT.DPNG.CD,DT.INT.MLAT.CD,DT.INT.OFFT.CD,DT.INT.PBND.CD,DT.INT.PCBK.CD,DT.INT.PROP.CD,DT.INT.PRVT.CD',
 'Afghanistan,AFG,61739336.9,100847181.9,,39107845,100847181.9,,,,,49114729.4,72894453.7,23779724.3,72894453.7,,,,39903620.1,53239440.1,,13335820,53239440.1,,,,',
 'Albania,ALB,54250280.6,790248675.2,514185620,182197616.7,236447897.3,0,39445139.5,170018.4,39615157.9,76050616.1,317194512.5,234321242.3,310371858.4,4542664.9,2279989.2,6822654.1,13847333.6,165602386.9,87884000,28101536.1,41948869.7,31030688.2,4618504.3,120324.7,35769517.2',
 'Algeria,DZA,95188724.6,171185188.1,75420000,0,95188724.6,,576463.5,0,576463.5,10320772.2,10320772.2,,10320772.2,,,,7680627.6,19031728.7,3220000,8094779,

In [5]:
first_row = deuda.take(1)[0]
deuda = deuda.filter(lambda x: x != first_row) #se realiza el mismo proceso de eliminar el encabezado
deuda = deuda.map(lambda x: x.split(",")) #se separan los datos a partir de las comas
deuda.take(5)

[['Afghanistan',
  'AFG',
  '61739336.9',
  '100847181.9',
  '',
  '39107845',
  '100847181.9',
  '',
  '',
  '',
  '',
  '49114729.4',
  '72894453.7',
  '23779724.3',
  '72894453.7',
  '',
  '',
  '',
  '39903620.1',
  '53239440.1',
  '',
  '13335820',
  '53239440.1',
  '',
  '',
  '',
  ''],
 ['Albania',
  'ALB',
  '54250280.6',
  '790248675.2',
  '514185620',
  '182197616.7',
  '236447897.3',
  '0',
  '39445139.5',
  '170018.4',
  '39615157.9',
  '76050616.1',
  '317194512.5',
  '234321242.3',
  '310371858.4',
  '4542664.9',
  '2279989.2',
  '6822654.1',
  '13847333.6',
  '165602386.9',
  '87884000',
  '28101536.1',
  '41948869.7',
  '31030688.2',
  '4618504.3',
  '120324.7',
  '35769517.2'],
 ['Algeria',
  'DZA',
  '95188724.6',
  '171185188.1',
  '75420000',
  '0',
  '95188724.6',
  '',
  '576463.5',
  '0',
  '576463.5',
  '10320772.2',
  '10320772.2',
  '',
  '10320772.2',
  '',
  '',
  '',
  '7680627.6',
  '19031728.7',
  '3220000',
  '8094779',
  '15775406.6',
  '',
  '13192.3'

##### 2. Número de países distintos en cada dataset. Coinciden?

In [ ]:
paises = deuda.map(lambda x: (x[0])) #se selecciona la primera columna del dataset "deuda", que indica el nombre de los paises

In [ ]:
paises.distinct().count() #se cuenta el número de distintos paises

123

In [ ]:
paises2 = desa_sep.map(lambda x: (x[0])) #se selecciona la primera columna del dataset "desa_sep", que indica el nombre de los paises

In [ ]:
paises2.distinct().count()

52

No coinciden el número de paises.


##### 3. Total de deuda contraida por cada pais: total amount of debt (in USD) DT.AMT.MLAT.CD

In [ ]:
deuda_por_pais = deuda.map(lambda x: (x[0], x[5])) #se seleccionan las columnas que hacen referencia al nombre de los paises y DT.AMT.MLAT.CD
deuda_por_pais.take(5) #se muestran las 5 primeras observaciones para comprobar que se ha realizado correctamente

[('Afghanistan', '39107845'),
 ('Albania', '182197616.7'),
 ('Algeria', '0'),
 ('Angola', '124688691.9'),
 ('Armenia', '94331207.1')]

##### 4. Media de los indicadores de deuda (tabla uno): DT.AMT.BLAT.CD, DT.DIS.BLAT.CD, DT.INT.BLAT.CD

In [ ]:
from pyspark.sql import SQLContext

In [ ]:
indicador1 = deuda.map(lambda x: x[2]) #se selecciona la columna que hace referencia a DT.AMT.BLAT.CD
indicador1=indicador1.filter(lambda x_int: x_int.isdigit())
indicador1=indicador1.map(lambda x: int(x))
indicador1.mean()


61416650.11111111

In [ ]:
indicador2 = deuda.map(lambda x: x[11]) #se selecciona la columna que hace referencia a DT.DIS.BLAT.CD
indicador2=indicador2.filter(lambda x_int: x_int.isdigit())
indicador2=indicador2.map(lambda x: int(x))
indicador2.mean()

506624979.33333343

In [ ]:
indicador3 = deuda.map(lambda x: x[18]) #se selecciona la columna que hace referencia a DT.INT.BLAT.CD
indicador3=indicador3.filter(lambda x_int: x_int.isdigit())
indicador3=indicador3.map(lambda x: int(x))
indicador3.mean()

53762351.333333336

##### 5. Los 20 paises con DT.AMT.DLXF.CD más alto

In [ ]:
top20 = deuda.map(lambda x: (float(x[3]),x[0])) #se seleccionan las columnas que hacen referencia a DT.AMT.DLXF.CD y al nombre de los paises
# utilizamos float para transformar los datos y poder operar con ellos
top20.top(20) #la funcion top mostrará los datos más elevados.


[(96218620835.7, 'China'),
 (90041840304.1, 'Brazil'),
 (66589761833.5, 'Russian Federation'),
 (51555031005.8, 'Turkey'),
 (48756295898.2, 'South Asia'),
 (31923507000.8, 'India'),
 (30916112653.8, 'Indonesia'),
 (27482093686.4, 'Kazakhstan'),
 (25218503927.0, 'Mexico'),
 (25197029299.4, 'Least developed countries: UN classification'),
 (20483289208.0, 'IDA only'),
 (14013783350.4, 'Romania'),
 (11985674438.7, 'Colombia'),
 (11067045628.1, 'Angola'),
 (10404814960.2, 'Cameroon'),
 (9506919669.6, 'Lebanon'),
 (9474257551.9, 'South Africa'),
 (8873505909.2, 'Vietnam'),
 (8336013891.3, 'Pakistan'),
 (8148995625.6, 'Ukraine')]

##### 6. Pais con los datos informados todos los años.

In [ ]:
todos_años = desa_sep.map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], False)
todos_años = todos_años.filter(lambda x: x[1] == 4) #se seleccionará el pais que se repita 4 veces (todos los años)
todos_años.take(10)

[('Malawi', 4)]

##### 7. Media anual de los distintos indicadores de desarrollo

Media de la primera variable:


In [ ]:
desarrollo1_2014 = desa_sep.filter(lambda x: x[2] == '2014 [YR2014]') #filtro que seleccionará los datos de 2014
desarrollo1_2014 = desarrollo1_2014.map(lambda x: x[3]) #se selecciona el primer indicador
desarrollo1_2014=desarrollo1_2014.filter(lambda x_int: x_int.isdigit()) #se transforman los datos para poder hacer la media
desarrollo1_2014=desarrollo1_2014.map(lambda x: int(x))
desarrollo1_2014.mean()


116610529707695.64

In [ ]:
desarrollo1_2015 = desa_sep.filter(lambda x: x[2] == '2015 [YR2015]') #filtro que seleccionará los datos de 2015
desarrollo1_2015 = desarrollo1_2015.map(lambda x: x[3]) #se selecciona el primer indicador
desarrollo1_2015=desarrollo1_2015.filter(lambda x_int: x_int.isdigit()) #se transforman los datos para poder hacer la media
desarrollo1_2015=desarrollo1_2015.map(lambda x: int(x))
desarrollo1_2015.mean()

184827281799554.75

In [ ]:
desarrollo1_2016 = desa_sep.filter(lambda x: x[2] == '2016 [YR2016]') #filtro que seleccionará los datos de 2016
desarrollo1_2016 = desarrollo1_2016.map(lambda x: x[3]) #se selecciona el primer indicador
desarrollo1_2016=desarrollo1_2016.filter(lambda x_int: x_int.isdigit()) #se transforman los datos para poder hacer la media
desarrollo1_2016=desarrollo1_2016.map(lambda x: int(x))
desarrollo1_2016.mean()

274875240137558.72

In [ ]:
desarrollo1_2017 = desa_sep.filter(lambda x: x[2] == '2017 [YR2017]') #filtro que seleccionará los datos de 2017
desarrollo1_2017 = desarrollo1_2017.map(lambda x: x[3]) #se selecciona el primer indicador
desarrollo1_2017.take(10)

['2239858619962.69']

Media de la segunda variable:


In [ ]:
desarrollo2_2014 = desa_sep.filter(lambda x: x[2] == '2014 [YR2014]') #filtro que seleccionará los datos de 2014
desarrollo2_2014 = desarrollo2_2014.map(lambda x: x[4]) #se selecciona el segundo indicador
desarrollo2_2014=desarrollo2_2014.map(lambda x: float(x)) #se transforman los datos para poder hacer la media
desarrollo2_2014.mean()

58.15214676864467

In [ ]:
desarrollo2_2015 = desa_sep.filter(lambda x: x[2] == '2015 [YR2015]') #filtro que seleccionará los datos de 2014
desarrollo2_2015 = desarrollo2_2015.map(lambda x: x[4]) #se selecciona el segundo indicador
desarrollo2_2015=desarrollo2_2015.map(lambda x: float(x)) #se transforman los datos para poder hacer la media
desarrollo2_2015.mean()

59.76292583345264

In [ ]:
desarrollo2_2016 = desa_sep.filter(lambda x: x[2] == '2016 [YR2016]') #filtro que seleccionará los datos de 2014
desarrollo2_2016 = desarrollo2_2016.map(lambda x: x[4]) #se selecciona el segundo indicador
desarrollo2_2016=desarrollo2_2016.map(lambda x: float(x)) #se transforman los datos para poder hacer la media
desarrollo2_2016.mean()

60.44188301621869

In [ ]:
desarrollo2_2017 = desa_sep.filter(lambda x: x[2] == '2017 [YR2017]') #filtro que seleccionará los datos de 2014
desarrollo2_2017 = desarrollo2_2017.map(lambda x: x[4]) #se selecciona el segundo indicador
desarrollo2_2017=desarrollo2_2017.map(lambda x: float(x)) #se transforman los datos 
desarrollo2_2017.take(1)

[48.659564238322]

Media de la tercera variable:

In [ ]:
desarrollo3_2014 = desa_sep.filter(lambda x: x[2] == '2014 [YR2014]') #filtro que seleccionará los datos de 2014
desarrollo3_2014 = desarrollo3_2014.map(lambda x: x[5]) #se selecciona el tercer indicador
desarrollo3_2014=desarrollo3_2014.filter(lambda x_int: x_int.isdigit()) #se transforman los datos para poder hacer la media
desarrollo3_2014=desarrollo3_2014.map(lambda x: int(x))
desarrollo3_2014.mean()

12.333333333333334

In [ ]:
desarrollo3_2015 = desa_sep.filter(lambda x: x[2] == '2015 [YR2015]') #filtro que seleccionará los datos de 2014
desarrollo3_2015 = desarrollo3_2015.map(lambda x: x[5])  #se selecciona el tercer indicador
desarrollo3_2015=desarrollo3_2015.filter(lambda x_int: x_int.isdigit()) #se transforman los datos para poder hacer la media
desarrollo3_2015=desarrollo3_2015.map(lambda x: int(x))
desarrollo3_2015.mean()

11.333333333333334

In [ ]:
desarrollo3_2016 = desa_sep.filter(lambda x: x[2] == '2016 [YR2016]') #filtro que seleccionará los datos de 2014
desarrollo3_2016 = desarrollo3_2016.map(lambda x: x[5])  #se selecciona el tercer indicador
desarrollo3_2016=desarrollo3_2016.filter(lambda x_int: x_int.isdigit()) #se transforman los datos para poder hacer la media
desarrollo3_2016=desarrollo3_2016.map(lambda x: int(x))
desarrollo3_2016.mean()

19.0

In [ ]:
desarrollo3_2017 = desa_sep.filter(lambda x: x[2] == '2017 [YR2017]') #filtro que seleccionará los datos de 2014
desarrollo3_2017 = desarrollo3_2017.map(lambda x: x[5])  #se selecciona el tercer indicador
desarrollo3_2017.take(10)

['34.593']

##### 8. Podrías decirme el total de deuda acumulada DT.AMT.MLAT.CD por los 10 países con un valor en media menor de SP.DYN.CBRT.IN

In [ ]:
nacimientos = desa_sep.map(lambda x: x[5])
nacimientos=nacimientos.filter(lambda x_int: x_int.isdigit()) #se transforman los datos para poder hacer la media
nacimientos=nacimientos.map(lambda x: int(x))
nacimientos.mean()

13.625

In [ ]:
natalidad_por_pais = desa_sep.filter(lambda x: x[2] == '2016 [YR2016]')
natalidad_por_pais = natalidad_por_pais.map(lambda x: (float(x[5]),x[0]))
natalidad_por_pais.takeOrdered(10, key = lambda x: x[0])

[(7.8, 'Japan'),
 (7.9, 'San Marino'),
 (8.8, 'Spain'),
 (9.4, 'Singapore'),
 (9.7, 'Hungary'),
 (10.3, 'Ukraine'),
 (10.445, 'Moldova'),
 (10.5, 'Switzerland'),
 (11.8, 'United Kingdom'),
 (12.0, 'Palau')]

De los 10 paises cuya tasa de natalidad es menor que la media, soo Ucrania y Moldavia aparecen en el dataset "deuda". Por lo tanto, se estudiará solamente la deuda acumulada de estos dos paises. 

In [12]:
Ukraine = deuda.filter(lambda x: 'Ukraine' in x[0])
deuda_acumulada = Ukraine.map(lambda x: (float(x[5]), x[0]))
deuda_acumulada.take(1)

[(1072472830.3, 'Ukraine')]

In [13]:
Moldova = deuda.filter(lambda x: 'Moldova' in x[0])
deuda_acumulada = Moldova.map(lambda x: (float(x[5]), x[0]))
deuda_acumulada.take(1)

[(135842865.5, 'Moldova')]

##### 9. ¿Hay alguna relación entre los nacimientos y el indicador DT.AMT.DLXF.CD? ¿Cómo podrías demostrarlo?